# RDDs + Programación Funcional

# Creamos un contexo para crear RDDs

In [30]:
import pandas as pd
!pip install pyspark --quiet
from pyspark import SparkContext
sc = SparkContext(master = "local", appName = "Transformaciones sobre un RDD") 

ValueError: ignored

# Cargamos un RDDs

Para la realzación del ejercicio, con ayuda del archivo paises, realizaremos el equivalente de operaciones 'select','count','group by' y 'filter / where'

Cambia el valor de la ruta para que apunte a la ruta donde tienes los datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
#LEER CSV
equiposOlimpicosRDD = sc.textFile("/paises.csv").map(lambda line : line.split(","))

In [ ]:
equiposOlimpicosRDD.take(10)

### Deshacemos un RDD

Con ayuda del método `collect()`, permite desparalelizar un RDD. 


In [ ]:
equiposOlimpicosRDD.collect()

Creamos un dataframe a partir de un RDD mediante `pd.DataFrame()` 

In [ ]:
pd.DataFrame(equiposOlimpicosRDD.collect()[1:], columns = equiposOlimpicosRDD.collect()[0] )

# Operaciones sobre un RDDs

Para la realzación del ejercicio, con ayuda del archivo paises, realizaremos el equivalente de operaciones `select`,`count`,`group by` , `filter` y `where`

### Ejercicio 1: 
**Mostrar los países sin repetidos**

*   Seleccionar la columna de las siglas PISTA: Usa un map y quedate con la columna con indice 2.
*   Usamos el `distinct()`



In [ ]:
#Inserta aquí tu código
equiposOlimpicosRDD.map(lambda fila : fila[2]).distinct().collect()

### Ejercicio 2
**Contar la cantidad de siglas diferentes de los equipos olímpicos existentes**

PISTA: Puedes usar `count()`

In [ ]:
#Inserta aquí tu código
equiposOlimpicosRDD.map(lambda fila : fila[2]).distinct().count()

### Ejercicio 3 (Díficil, puedes hacerlo el último)
**Agrupamos datos para poder determinar cuantos equipos posee un pais**

Recuerda que los RDDs poseen una estructura de `clave-valor`, por lo cual debemos poner primero el valor 'clave' (la sigla del país).

Con 'mapValues', al componente 'valor' le indicamos que operación deseamos que se le aplique.

**Objetivo 1**: Tener un diccionario con `clave` la sigla del país y `valor` la lista de equipos de ese país

Con `groupByKey` podemos agrupar por `clave`, en este caso por sigla del país  

In [32]:
#Inserta aquí tu código
equiposOlimpicosRDD.map(lambda fila : fila[2]).groupByKey().mapValues(list)

PythonRDD[22] at RDD at PythonRDD.scala:53

**Objetivo 2**: Tener un diccionario con `clave` la sigla del país y `valor` la cantidad de equipos de ese país

In [35]:
#Inserta aquí tu código
resultado = equiposOlimpicosRDD.map(lambda fila : fila[2]).groupByKey().mapValues(len).collect()
resultado.map(lambda fila: fila[0], len(fila[1]) )).takeOrdered(3 lambda)

SyntaxError: ignored

### Operación filter para obtener un subconjunto

Con el método 'filter', reducimos el conjuntos de equipos.

Nos quedamos con la sigla equivalente de argentina

In [ ]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l : "ARG" in l)
equiposArgentinos.collect()  

# Accciones sobre RDDs

In [ ]:
deportistaOlimpicoRDD = sc.textFile("/content/drive/MyDrive/Colab Notebooks/RED.ES/Datos Ejercicios/M6/deportista.csv").map(lambda line : line.split(","))
deportistaOlimpico2RDD = sc.textFile("/content/drive/MyDrive/Colab Notebooks/RED.ES/Datos Ejercicios/M6/deportista2.csv").map(lambda line : line.split(","))

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpico2RDD)

### Formas de visualizar datos de un RDDs

La operación 'take' nos devuelve 'N' valores que encuentre spark.

La operación 'top', previo ordena respecto al valor llave y nos devuelve 'N' valores.

La operacion 'takeSample', nos devuelve una muestra aleatoria de los valores, Observa que recibe tres parametros

| Orden | Argumento | Descripción | Valor
|-------|--------|-----|--------|
|1|withReplacement|Indica si la muetra podrá traer replicados|Bool|
|2| num| Cantidad de valores a retornar|int|
|3|seed|semilla para el generador aleatorio|int|

Nota: Si encuentras complicado leer el código en los segmentos donde usamos indices en las listas, ejecutalo por partes para que visualizes que componentes seleccionamos 

Nota: Para hacer join con RDDs, debemos tener selecionada la llave al inicio del RDD para poder realizar el cruce.

In [ ]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                        .map(lambda x : [x[0],x[2]])).take(6)

In [ ]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                        .map(lambda x : [x[0],x[2]])).top(5)

In [ ]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                        .map(lambda x : [x[0],x[2]])).takeSample(False,4,10)

### Ejercicio 4
**Muestra 17 filas del RDD deportistaOlimpicoRDD**


In [12]:
#Inserta aquí tu código
deportistaOlimpicoRDD.take(17)

NameError: ignored

## Importancia de countAprox

Debido a la cantidad de datos no siempre es recomendable hacer operaciones tipo count.

Por lo que 'countAprox' es la solución mas viable cuando solo queremos darnos una idea de cuantos datos podemos leer durante un tiempo determinado. Nota: el parametro está en milisegundos

In [13]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                            .map(lambda x : [x[0],x[2]])).count()

NameError: ignored

In [ ]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                        .map(lambda x : [x[0],x[2]])).countApprox(20)

# Acciones de modificacion

### Obtenemos el equipo y el deportista

Guardamos el RDD resultante en equipoDeportista

In [ ]:
equipoDeportista = deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                                            .map(lambda x : [x[0],x[2]]))

In [ ]:
equipoDeportista.take(1)


In [ ]:
equipoDeportista.map(lambda x : (x[1][0][0],x[1][0][1:],x[1][1]) )

### Carga de resultados

Esta tabla posee las medallas que los jugadores han ganado

In [ ]:
resultado = sc.textFile("/content/drive/MyDrive/Colab Notebooks/RED.ES/Datos Ejercicios/M6/resultados.csv").map(lambda line : line.split(","))

Eliminamos todas las medallas no ganadoras y nos quedamos con el valor de la medalla y  del deportista_id

In [ ]:
resultadoGanador = resultado.filter(lambda l : 'NA' not in l[1])
resultadoGanador = resultadoGanador.map(lambda l : [l[2],l[1]])

In [ ]:
resultadoGanador.take(15)

### Obtenemos la relación buscada: deportista,pais y medalla.

In [ ]:
jugadoresMedalla =  equipoDeportista.join(resultadoGanador)
jugadoresMedalla.take(1)

[('514', ((['362', 'Isao Ko Abe', '1', '24', '177', '75'], 'JPN'), 'Silver'))]

### Agrupamos las medallas respecto a la sigla del pais jugador

In [ ]:
d = {'Gold':7, 'Silver':5, 'Bronze':4}
paisesMedallas = jugadoresMedalla.map(lambda x : (x[1][0][-1],d[x[1][1]]) )
paisesMedallas.takeSample(False,10)

### Obtenemos los valores de los puntuajes históricos de los paises jugadores

In [ ]:
from operator import add
conclusion = paisesMedallas.reduceByKey((add)).sortBy(lambda x : x[1],ascending=False)
conclusion.take(10)

[('ITA', 38559),
 ('NED', 29508),
 ('GER', 11130),
 ('JPN', 7880),
 ('NZL', 4646),
 ('TCH', 3880),
 ('TUR', 2315),
 ('RSA', 2175),
 ('BLR', 2156),
 ('ALG', 2024)]

# Estadística básica sobre los RDDs

Spark posee una suite integrada de forma natural para poder obtener estadísticas básicas.

In [ ]:
conclusion.map(lambda l : l[1]).stats()

In [ ]:
conclusion.map(lambda l : l [1]).mean()

In [ ]:
conclusion.map(lambda l : l [1]).sum()

In [ ]:
conclusion.map(lambda l : l [1]).histogram(10)

## Ejercicios Programación funcional

### Ejercicio 5
**Escribir una función que aplique el IVA a un precio.**
* Nombre de la función: apply_iva
* Argumentos: precio, iva
* Resultado: precio con iva aplicado


* Prueba a llamar a la función


In [14]:
#Inserta aquí tu código
def apply_iva(precio,iva):
  total = (iva * precio / 100) + precio
  return total

In [15]:
apply_iva(98,21)

118.58

### Ejercicio 6
** Escribe una función que aplica la función cuadrado() a todos los elementos de una lista.**

    Parámetros:
        funcion: Recibe la función a aplicar.
        lista: Es una lista con valores que se pasarán como argumentos a funcion.
    Devuelve:
        Una lista con el resultado de aplicar la función a los valores de la lista.

In [20]:
def cuadrado(n):
    return n * n

In [ ]:
print(aplica_funcion_lista(cuadrado, [1, 2, 3, 4]))

In [19]:
#Inserta aquí tu código
def applicar_funcion_lista(funcion, lista):
  for numero in lista:
    cuadrado(numero)

### Ejercicio 7
**Detectar y corregir los errores del siguiente programa que aplica el iva a una factura:**


In [24]:
base = float(input('Introduce la base imponible de la factura: '))
iva = int(input('Introduce el IVA de la factura: '))

def aplica_iva(base, iva = 21):
    base = base * iva / 100 + base  
    return base

print(aplica_iva(base, iva))

Introduce la base imponible de la factura: 88
Introduce el IVA de la factura: 21
106.48
